# Cryospheric Model Comparison Tool (CmCt) - Altimetry Module

The Cryosphere model Comparison tool (CmCt) is an ice sheet model validation tool that has been developed by NASA to facilitate direct comparison between satellite observational data and various ice sheet models. The CmCt allows the user to take advantage of several decades worth of satellite-based observations from Greenland.

The Cryospheric Model Comparison Tool (CmCt) is designed to facilitate rapid comparison between ice sheet model results, and between ice sheet models and available observations. The observational data sets available through the CmCt are processed and edited following community best practices (such as those used by the IMBIE2 effort), and eliminate the need for detailed understanding of remote sensing data by producing data sets that can be directly compared with model output variables. The overall goal of the CmCt is to eliminate barriers preventing the use of remote sensing data by the ice sheet modeling community. 

The CmCt accepts input models of surface elevations over Greenland. Currently, the CmCt is used to compare ice sheet models provided by the user with remotely sensed satellite ICESat (Ice, Cloud, and land Elevation Satellite) laser altimetry, ERS-1, ERS-2 (European Remote-Sensing Satellite), and Envisat (Environmental Satellite) radar altimetry data.

## Model Information

**Input Model:** The models MUST be defined on a rectangular X-Y grid in the ISMIP6 standard projected polar-stereographic space. (Note, NOT a lat-lon grid!) The ISMIP6 standard projection is defined [here](https://www.climate-cryosphere.org/wiki/index.php?title=ISMIP6_wiki_page). However, any grid spacing may be used. For internal consistency, we don't use the model X and Y values, but reproject the model latitudes and longitudes. Grid cell nodes are in the upper-left corner of the cell.

**Format:** There is only netCDF model input format accepted. The netCDF files MUST include 2-D (x,y) latitude and longitude variables and these variables MUST have "standard_name" attributes with values "latitude" and "longitude" respectively; because the model is on an x-y grid, latitude and longitude will vary along rows and columns. A sample of a netCDF file accepted by the CmCt can be downloaded from here.

## Running the Tool

* Create Run Directory Structure
* Configure Run
* Write Configuration File
* Launch CmCt Altimetry Module Backend

## CmCt Results

**CmCt Outputs**: 
* Output Log File: This file will contain metadata about the run, messages about the run’s progression, lists of the individual data files used in the processing, and some statistics about the comparison. At the end of each log file there will be statistics for the entire input grid as a whole. In addition, any error or warning messages will be in this file.
* Record List File: This is the main output from the tool. This file begins with a detailed header that gives metadata about the input parameters to the analysis, and explains the columns in the file. All the header lines begin with the character "#". This file contains a record for each sensor footprint that was within the model boundary.
* Mean and Standard Deviation Grids File: For every cell defined by 4 adjacent grid nodes of the input model, this file contains statistics on all the elevation deltas within that cell. Calculations for this file are done using a non-standard projection. 
* Histogram File: This file contains the 1 meter histogram of the elevation deltas, their probability density function, and their empirical cumulative distribution function. It begins with a header similar to that of the record list file, but which also includes histogram statistics. The histogram file plots of Cumulative Distribution and Probability Distribution function can be viewed with the CmCt Plotting tool.
* JSON Configuration Files: Files with the .json extension are configuration files. These files contain metadata about the model run, data input, and fields chosen from the web interface. These files will be useful to troubleshoot the model run if there are problems during the process.


**Altimetry Mission:** Select the altimetry dataset for comparison. Currently, the available options are ICESat laser altimetry, ERS-1, ERS-2, and Envisat radar altimetry data.

**Data set:** Select the dataset to compare your model against. There are 3 available data sets: \
&nbsp;•	WGS84 elevation – for Greenland or Antarctica (Topex/Poseidon elevation) is a preprocessed subset of elevations on the TOPEX/Poseidon ellipsoid. \
&nbsp;•	EGM08 Mean Tide elevation – for Greenland or Antarctica is a preprocessed subset of elevations on the EGM08 Mean Tide reference system. \
&nbsp;•	EGM08 Tide-Free elevation – for Greenland or Antarctica is a preprocessed subset of elevations on the EGM08 Tide-Free reference system.


**Grid Spacing:** Enter the grid spacing of the input model in kilometers (km). Do not include units. The value can be a real or an integer. Regardless, the software uses a real value in its calculations.

**Model time index:** Enter an integer value for the desired time step. Indexing begins at 1. Note that this is an INDEX, and NOT the time value itself. The input models should have (x,y,t) coordinates, and this enables a user to select which time "slice" to analyze.

## Import Python Packages

In [1]:
# Imports
import os,sys
import glob as glob
import json
import shutil
import subprocess
from datetime import datetime

import ipywidgets as widgets
from IPython.display import display,clear_output
from ipywidgets import HBox, VBox

from CMCT_select_data_parameters import *
import time



In [2]:
# Start timing
start_time = time.time()

## Set up Run Configuration Name and Folder

In [3]:
# Working Directory
workingdir = os.getcwd()

# Set path to upload directory
# upload_dir = "/CmCt/cmct_input_files/"
# upload_dir = os.path.expanduser('~/CmCt/cmct_input_files/')
upload_dir = os.path.expanduser('/home/jovyan/CmCt/RUNS/')
#Data dir and model name
# Datadir = "/CmCt/"
# Datadir =os.path.expanduser('~/CmCt/')
Datadir =os.path.expanduser('/home/jovyan/CmCt/')
datafile = "test_model_file.nc"

### Select Parameters

In [4]:
parameters = CMCT_select_data_parameters()
print(parameters)

{'region': 'greenland', 'mission': 'icesat-glas', 'dataset': 'icesat-cmctnc-tpelev-grn', 'year': '2003', 'variable': 'orog', 'spacing': '5.0', 'loginname_widget': 'rbasnet', 'email': 'ritu.basnet@nasa.gov', 'model_name': 'DenisTest-2024Jan03', 'actual_user_name': 'Ritu Basnet', 'model_time_index': '2'}


In [5]:
# Construct run_id using username and current time
date = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
date2 = datetime.now().strftime('%Y%m%dT%H%M%S')

run_id = parameters['loginname_widget']+ "_" + date2
# print(run_id)

# Create upload dir path
upload_dir_act = upload_dir + run_id + "/"
print(upload_dir_act)

#Make upload directory
os.mkdir(upload_dir_act)

# Move Model File to Run Directory
shutil.copy2(Datadir + datafile, upload_dir_act)

/home/jovyan/CmCt/RUNS/rbasnet_20240409T180900/


'/home/jovyan/CmCt/RUNS/rbasnet_20240409T180900/test_model_file.nc'

## Write Run Configuration File

In [6]:
# Write directly to upload dir
with open(upload_dir_act + 'cmct_run_config' + '_' + run_id + '.json', 'w') as outfile:
    
    json.dump({
        "cmct_run_config" : {
            "run" : {
                "runid" : run_id,
                "date" : date,
                "upload_dir" : upload_dir_act,
                "user_run_title" : parameters['model_name'],
                "loginname" : parameters['loginname_widget'],
                "actualusername" : parameters['actual_user_name'],
                "email" : parameters['email']
            },
            "comparisons" : [
                {
                    "model" : {
                        "modelname" : parameters['model_name'],
                        "filename" : datafile,
                        "format" : "netcdf",
                        "spacing_km" : float(parameters['spacing']),
                        "region" : parameters['region'],
                        "variable" : parameters['variable'],
                        "user_comments" : "",
                        "model_time_index" : int(parameters['model_time_index'])
                    },
                    "observations" : {
                        "mission" : parameters['mission'],   
                        "campaign" : parameters['year'],       
                        "dataset" : parameters['dataset']   
                    }
                }
            ]
        }
    }, outfile, indent=4, sort_keys=True)



## Launch CmCt Backend

In [7]:
# Construct path to launch script
case = "ICESAT"
name = "cmct_launch.ksh"
run_script = os.path.join(f'/home/jovyan/CmCt/CMCT_{case}', name)
print(run_script)

config_file = os.path.join(upload_dir_act, f'cmct_run_config_{run_id}.json')
print(config_file)

/home/jovyan/CmCt/CMCT_ICESAT/cmct_launch.ksh
/home/jovyan/CmCt/RUNS/rbasnet_20240409T180900/cmct_run_config_rbasnet_20240409T180900.json


Initiates CmCt Altimetry Module Backend and generates all comparison result files.

In [8]:
!sh "{run_script}" "{config_file}"

trap: ERR: bad trap
mkdir: cannot create directory ‘/home/jovyan/CmCt/RUNS/rbasnet_20240409T180900’: File exists
trap: ERR: bad trap
trap: ERR: bad trap
CMCT_rbasnet_20240409T180900/
CMCT_rbasnet_20240409T180900/cmct_run_config_rbasnet_20240409T180900.json
CMCT_rbasnet_20240409T180900/CMCT_rbasnet_20240409T180900.cmct_main.log
CMCT_rbasnet_20240409T180900/icesat_cism_config.json
CMCT_rbasnet_20240409T180900/datasets.json
CMCT_rbasnet_20240409T180900/icesat_cmctnc_grn_config.json
CMCT_rbasnet_20240409T180900/cmct_main_control_rbasnet_20240409T180900.json


In [9]:
# End timing
end_time = time.time()
execution_time = end_time - start_time

# Print execution time
print(f"Execution time: {execution_time} seconds")
      
# 282.44801020622253 seconds = 4.71 minutes 2003
# 1815.4350416660309 seconds= 30.26 minutes minutes 2004

Execution time: 376.94930124282837 seconds
